In [1]:
import sklearn  # collection of machine learning algorithms
from IPython import display  # pretty printing of dataframes in Jupyter notebook
# collection of functions for scientific computing and advance mathematics
import scipy as sp
import numpy as np  # foundational package for scientific computing
# collection of functions for scientific and publication-ready visualization
import matplotlib
import pandas as pd  # collection of functions for data processing and analysis modeled after R dataframes with SQL like features
import pandas as pd
import sweetviz as sv
import seaborn as sns
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import metrics
from sklearn import model_selection
from sklearn import feature_selection
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from xgboost import XGBClassifier
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from subprocess import check_output
import warnings
import time
import random
import IPython
import sys  # access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))

print("SciPy version: {}". format(sp.__version__))

print("IPython version: {}". format(IPython.__version__))

print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries


#ignore warnings
warnings.filterwarnings('ignore')
print('-'*25)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#misc libraries


#Common Model Helpers

#Visualization

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline


Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
SciPy version: 1.7.1
IPython version: 7.26.0
scikit-learn version: 0.24.2
-------------------------


In [33]:
#pathからcsvファイルを読み込む場合
data_raw = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_train = data_raw.copy(deep=True)
data_cleaner = [df_train, df_test]
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print("--------------------")
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print("--------------------")

Shape of Train Data
[row :891,column :12]
--------------------
Shape of Test Data
[row :418,column :11]
--------------------


In [34]:
for dataset in data_cleaner:
    #Discrete variables
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

    dataset['IsAlone'] = 1  # initialize to yes/1 is alone
    # now update to no/0 if family size is greater than 1
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    dataset['Title'] = dataset['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 30)


In [35]:
label = LabelEncoder()
for dataset in data_cleaner:
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])
    dataset['Title_Code'] = label.fit_transform(dataset['Title'])
    dataset['FareBin_Code'] = label.fit_transform(dataset['FareBin'])


In [36]:
df_train['Age'][df_train['Title'] == "Mr"].fillna(df_train['Age'][df_train['Title'] == "Mr"].mean(),inplace=True)
df_train['Age'].isnull().sum()

177

In [12]:
dataset = [df_train,df_test]
for dataset in data_cleaner:
    for i in dataset["Title"].unique():
        print(i)
        dataset['Age'][dataset['Title'] == i].fillna(dataset['Age'][dataset["Title"] == i].mean(), inplace=True)

    # dataset["Age"].fillna(dataset["Age"].mean(),inplace=True)
    dataset['AgeBin'] = pd.qcut(dataset['Age'], 3)
    dataset['Age_Code'] = label.fit_transform(dataset['AgeBin'])


Mr
Mrs
Miss
Master
Don
Rev
Dr
Mme
Ms
Major
Lady
Sir
Mlle
Col
Capt
the Countess
Jonkheer
Mr
Mrs
Miss
Master
Ms
Col
Rev
Dr
Dona


In [13]:
# repoort = sv.analyze(df_train,target_feat="Perished")
# repoort.show_html("SWEETVIZ_REPORT.html")
df_train.isnull().sum()

PassengerId        0
Perished           0
Pclass             0
Name               0
Sex                0
Age              177
SibSp              0
Parch              0
Ticket             0
Fare               0
Cabin            687
Embarked           2
FamilySize         0
IsAlone            0
Title              0
FareBin            0
Sex_Code           0
Embarked_Code      0
Title_Code         0
FareBin_Code       0
AgeBin           177
Age_Code           0
dtype: int64

In [118]:
#save csv
import sys
import os
PATH_dir = "./df"
df_train.to_csv(os.path.join(PATH_dir, "EDA_09_GBDT_train.csv"), index_label=False)
df_test.to_csv(os.path.join(PATH_dir, "EDA_09_GBDT_test.csv"), index_label=False)
